<a href="https://colab.research.google.com/github/anthonyibr24/Feature-Selection-Methods/blob/Magic-Telescope/Magic_Telescope.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ucimlrepo

In [ ]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np

# fetch dataset
magic_gamma_telescope = fetch_ucirepo(id=159)

# data (as pandas dataframes)
X = magic_gamma_telescope.data.features
y = magic_gamma_telescope.data.targets

df=pd.concat([X,y],axis=1)
df.head()
print(df['class'].value_counts())



class
g    12332
h     6688
Name: count, dtype: int64


In [ ]:
df.shape

(19020, 11)

In [ ]:
print(df['class'].value_counts())

class
g    12332
h     6688
Name: count, dtype: int64


In [ ]:
from sklearn.utils import Bunch

# Create a Bunch-like object for compatibility
data_set = Bunch(
    data=df.drop('class', axis=1).values,
    target=df['class'].values,
    feature_names=df.drop('class', axis=1).columns.tolist(),
)


#load dataset
data=data_set
df=pd.DataFrame(data.data,columns=data.feature_names)
df['target']=data.target
df.head()

#scale mean=0 std=1
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
df=df.drop('target',axis=1)
df[data.feature_names]=scaler.fit_transform(df[data.feature_names])

df['target']=data.target
df.describe().round(3)

,fLength,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist
count,19020.000,19020.000,19020.000,19020.000,19020.000,19020.000,19020.000,19020.000,19020.000,19020.000
mean,0.000,-0.000,-0.000,-0.000,0.000,-0.000,0.000,-0.000,0.000,-0.000
std,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
min,-1.156,-1.209,-1.870,-2.009,-1.940,-7.661,-6.712,-9.898,-1.059,-2.576
25%,-0.683,-0.562,-0.736,-0.791,-0.780,-0.275,-0.459,-0.533,-0.847,-0.687
50%,-0.380,-0.275,-0.181,-0.143,-0.164,0.141,0.094,0.020,-0.382,-0.026
75%,0.398,0.139,0.585,0.675,0.639,0.480,0.496,0.514,0.699,0.626
max,6.631,12.766,5.286,2.804,4.168,9.789,4.466,8.624,2.389,4.038


# Full features

In [ ]:
#Cross validation
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression

lr=LogisticRegression()

cv_results = cross_validate(
    lr,
    df.drop('target', axis=1),
    df['target'],
    cv=4,
    scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'],
    return_train_score=False,

)

print("\nAverage Metrics Across 4 Folds:")
print(f"Accuracy:  {np.mean(cv_results['test_accuracy']):.4f} ± {np.std(cv_results['test_accuracy']):.4f}")
print(f"Precision: {np.mean(cv_results['test_precision_macro']):.4f} ± {np.std(cv_results['test_precision_macro']):.4f}")
print(f"Recall:    {np.mean(cv_results['test_recall_macro']):.4f} ± {np.std(cv_results['test_recall_macro']):.4f}")
print(f"F1 Score:  {np.mean(cv_results['test_f1_macro']):.4f} ± {np.std(cv_results['test_f1_macro']):.4f}")


Average Metrics Across 4 Folds:
Accuracy:  0.7908 ± 0.0022
Precision: 0.7813 ± 0.0040
Recall:    0.7453 ± 0.0031
F1 Score:  0.7567 ± 0.0027


# Using 13 features


---


Accuracy:  0.9777


Precision: 0.9773


Recall:    0.9806


F1 Score:  0.9781


---



# Mutual Gain

In [ ]:
#Mutual gain
from sklearn.datasets import load_wine
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

#load dataset
data=data_set
df=pd.DataFrame(data.data,columns=data.feature_names)
df['target']=data.target
df.head()

X=df.drop('target',axis=1)
y=df['target']



In [ ]:
#Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SelectKBest(mutual_info_classif, k=5)),
    ('classifier', LogisticRegression())
])
pipeline.verbose=True

#Cross-validate and calculate all metrics
scores = cross_validate(
    pipeline,
    X, y,
    cv=4,
    scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
)

print("\nAverage Scores Across 4 Folds:")
for metric in ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']:
    mean_score = scores[f'test_{metric}'].mean()
    std_score = scores[f'test_{metric}'].std()
    print(f"{metric.capitalize()}: {mean_score:.4f} ± {std_score:.4f}")

[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.8s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.8s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   1.0s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.1s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   1.2s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.1s

Average Scores Across 4 Folds:
Accuracy: 0.7907 ± 0.0012
Precision_macro: 0.7829 ± 0.0026
Recall_macro: 0.7433 ± 0.0033
F1_macro: 0.7552 ± 0.0025


In [ ]:
#ANOVA
from sklearn.datasets import load_wine
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline


#load dataset
data=data_set
df=pd.DataFrame(data.data,columns=data.feature_names)
df['target']=data.target
df.head()

X=df.drop('target',axis=1)
y=df['target']

#Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SelectKBest(score_func=f_classif, k=5)),
    ('classifier', LogisticRegression())
])
pipeline.verbose=True

#Cross-validate and calculate all metrics
scores = cross_validate(
    pipeline,
    X, y,
    cv=4,
    scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
)

print("\nAverage Scores Across 4 Folds:")
for metric in ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']:
    mean_score = scores[f'test_{metric}'].mean()
    std_score = scores[f'test_{metric}'].std()
    print(f"{metric.capitalize()}: {mean_score:.4f} ± {std_score:.4f}")

[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.0s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.1s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.0s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.1s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.0s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.1s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.0s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.1s

Average Scores Across 4 Folds:
Accuracy: 0.7905 ± 0.0011
Precision_macro: 0.7825 ± 0.0022
Recall_macro: 0.7432 ± 0.0031
F1_macro: 0.7552 ± 0.0024


# Wrapper Methods

# Forward

In [ ]:
#FORWARD
from sklearn.datasets import load_wine
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier

#load dataset
data=data_set
df=pd.DataFrame(data.data,columns=data.feature_names)
df['target']=data.target
df.head()

X=df.drop('target',axis=1)
y=df['target']

knn=KNeighborsClassifier(5)



In [ ]:
#Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SequentialFeatureSelector(knn, n_features_to_select=5, direction='forward', scoring='accuracy')),
    ('classifier', LogisticRegression())
])
pipeline.verbose=True

#Cross-validate and calculate all metrics
scores = cross_validate(
    pipeline,
    X, y,
    cv=4,
    scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
)

print("\nAverage Scores Across 4 Folds:")
for metric in ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']:
    mean_score = scores[f'test_{metric}'].mean()
    std_score = scores[f'test_{metric}'].std()
    print(f"{metric.capitalize()}: {mean_score:.4f} ± {std_score:.4f}")



[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=  43.8s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.1s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=  40.9s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=  42.3s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=  42.3s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s

Average Scores Across 4 Folds:
Accuracy: 0.7889 ± 0.0013
Precision_macro: 0.7809 ± 0.0024
Recall_macro: 0.7408 ± 0.0037
F1_macro: 0.7528 ± 0.0029


# BACKWARD

In [ ]:
#BACKWARD
from sklearn.datasets import load_wine
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier

#load dataset
data=data_set
df=pd.DataFrame(data.data,columns=data.feature_names)
df['target']=data.target
df.head()

X=df.drop('target',axis=1)
y=df['target']

knn=KNeighborsClassifier(5)

In [ ]:
#Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SequentialFeatureSelector(knn, n_features_to_select=5, direction='backward', scoring='accuracy')),
    ('classifier', LogisticRegression())
])
pipeline.verbose=True

#Cross-validate and calculate all metrics
scores = cross_validate(
    pipeline,
    X, y,
    cv=4,
    scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
)

print("\nAverage Scores Across 4 Folds:")
for metric in ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']:
    mean_score = scores[f'test_{metric}'].mean()
    std_score = scores[f'test_{metric}'].std()
    print(f"{metric.capitalize()}: {mean_score:.4f} ± {std_score:.4f}")


[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=  59.7s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=  57.2s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total= 1.1min
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total= 1.1min
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s

Average Scores Across 4 Folds:
Accuracy: 0.7887 ± 0.0013
Precision_macro: 0.7802 ± 0.0024
Recall_macro: 0.7412 ± 0.0038
F1_macro: 0.7530 ± 0.0030


# RFE

In [ ]:
#RFE
from sklearn.datasets import load_wine
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE
from sklearn.svm import SVC

#load dataset
data=data_set
df=pd.DataFrame(data.data,columns=data.feature_names)
df['target']=data.target
df.head()

X=df.drop('target',axis=1)
y=df['target']



In [ ]:
#Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', RFE(estimator=SVC(kernel='linear'), n_features_to_select=5)),
    ('classifier', LogisticRegression())
])
pipeline.verbose=True

#Cross-validate and calculate all metrics
scores = cross_validate(
    pipeline,
    X, y,
    cv=4,
    scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
)

print("\nAverage Scores Across 4 Folds:")
for metric in ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']:
    mean_score = scores[f'test_{metric}'].mean()
    std_score = scores[f'test_{metric}'].std()
    print(f"{metric.capitalize()}: {mean_score:.4f} ± {std_score:.4f}")


[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=  20.1s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=  19.4s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=  18.9s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=  20.3s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s

Average Scores Across 4 Folds:
Accuracy: 0.7892 ± 0.0002
Precision_macro: 0.7793 ± 0.0025
Recall_macro: 0.7437 ± 0.0026
F1_macro: 0.7549 ± 0.0016


# Embeded Methods

# Random Forest

In [ ]:
#RF
from sklearn.datasets import load_wine
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel


#load dataset
data=data_set
df=pd.DataFrame(data.data,columns=data.feature_names)
df['target']=data.target
df.head()

X=df.drop('target',axis=1)
y=df['target']


In [ ]:
#Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SelectFromModel(RandomForestClassifier(random_state=42), max_features=5)),
    ('classifier', LogisticRegression())
])
pipeline.verbose=True

#Cross-validate and calculate all metrics
scores = cross_validate(
    pipeline,
    X, y,
    cv=4,
    scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
)

print("\nAverage Scores Across 4 Folds:")
for metric in ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']:
    mean_score = scores[f'test_{metric}'].mean()
    std_score = scores[f'test_{metric}'].std()
    print(f"{metric.capitalize()}: {mean_score:.4f} ± {std_score:.4f}")

[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   5.6s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   5.0s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   5.3s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   5.1s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s

Average Scores Across 4 Folds:
Accuracy: 0.7856 ± 0.0018
Precision_macro: 0.7767 ± 0.0033
Recall_macro: 0.7374 ± 0.0040
F1_macro: 0.7491 ± 0.0033


# Logistic using L1 penalty

In [ ]:
#LR L1 penalty
from sklearn.datasets import load_wine
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest


#load dataset
data=data_set
df=pd.DataFrame(data.data,columns=data.feature_names)
df['target']=data.target
df.head()

X=df.drop('target',axis=1)
y=df['target']


In [ ]:
#Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SelectFromModel(LogisticRegression(penalty='l1',solver='saga',C=0.1,max_iter=3000,random_state=42),max_features=5)),
    ('classifier', LogisticRegression())
])
pipeline.verbose=True

#Cross-validate and calculate all metrics
scores = cross_validate(
    pipeline,
    X, y,
    cv=4,
    scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
)

print("\nAverage Scores Across 4 Folds:")
for metric in ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']:
    mean_score = scores[f'test_{metric}'].mean()
    std_score = scores[f'test_{metric}'].std()
    print(f"{metric.capitalize()}: {mean_score:.4f} ± {std_score:.4f}")

[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.2s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.1s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.1s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.1s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.1s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s

Average Scores Across 4 Folds:
Accuracy: 0.7909 ± 0.0019
Precision_macro: 0.7819 ± 0.0034
Recall_macro: 0.7450 ± 0.0040
F1_macro: 0.7565 ± 0.0031
